In [ ]:
!pip install pandas

In [ ]:
!pip install certifi


In [ ]:
import json
import pandas as pd
import certifi
from urllib.request import urlopen  # or from urllib2 import urlopen (for older Python versions)


In [ ]:
try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen

import certifi
import json
import pandas as pd


def get_jsonparsed_data(url, api_key, exchange):
  if exchange == "NSE":
    url = f"https://financialmodelingprep.com/api/v3/search?query={ticker}&exchange=NSE&apikey={api_key}"
  elif exchange == "BSE":
        url = f"https://financialmodelingprep.com/api/v3/search?query={ticker}&exchange=BSE&apikey={api_key}"
  else:
    url = f"https://financialmodelingprep.com/api/v3/quote/{ticker}?apikey={api_key}"
  response = urlopen(url, cafile=certifi.where())
  data = response.read().decode("utf-8")
  return json.loads(data)

api_key="your_api_key"
ticker = "MSFT"
exchange = "US"
eco_ind = pd.DataFrame(get_jsonparsed_data(ticker, api_key,exchange))
eco_ind

<ipython-input-39-892d5aef57b0>:18: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


,symbol,name,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,...,exchange,volume,avgVolume,open,previousClose,eps,pe,earningsAnnouncement,sharesOutstanding,timestamp
0,MSFT,Microsoft Corporation,354.56,-0.92215,-3.3,350.3,373.65,468.35,344.79,2635791948800,...,NASDAQ,34995691,24604840,368.255,357.86,12.42,28.55,2025-04-23T20:00:00.000+0000,7433980000,1744142401


In [ ]:
!pip install langchain langchain-community langchain-core transformers


In [ ]:
!pip install langchain-community

In [ ]:
!pip install langchain-core

In [ ]:
!pip install transformers

In [ ]:
def preprocess_economic_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['earningsAnnouncement'] = pd.to_datetime(df['earningsAnnouncement'])
    return df

preprocessed_economic_df = preprocess_economic_data(eco_ind)
preprocessed_economic_df

,symbol,name,price,changesPercentage,change,dayLow,dayHigh,yearHigh,yearLow,marketCap,...,exchange,volume,avgVolume,open,previousClose,eps,pe,earningsAnnouncement,sharesOutstanding,timestamp
0,MSFT,Microsoft Corporation,354.56,-0.92215,-3.3,350.3,373.65,468.35,344.79,2635791948800,...,NASDAQ,34995691,24604840,368.255,357.86,12.42,28.55,2025-04-23 20:00:00+00:00,7433980000,1970-01-01 00:00:01.744142401


In [ ]:
preprocessed_economic_df.to_csv("eco_ind.csv")

In [ ]:
%pip install --upgrade --quiet  langchain sentence_transformers

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
hg_embeddings = HuggingFaceEmbeddings()

<ipython-input-48-733749dc74cb>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()


In [ ]:
hg_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader_eco = CSVLoader('eco_ind.csv')
documents_eco = loader_eco.load()

# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)

# Split your docs into texts
texts_eco = text_splitter.split_documents(documents_eco)

# Embeddings
embeddings = HuggingFaceEmbeddings()

<ipython-input-50-3472f2a463cb>:13: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
from langchain.vectorstores import Chroma

persist_directory = 'docs/chroma_rag/'

In [ ]:
!pip install chromadb

In [ ]:

import chromadb
print(chromadb.__version__)

1.0.3


In [ ]:
persist_directory = "./chroma_db"  # Change path as needed

In [ ]:
economic_langchain_chroma = Chroma.from_documents(
    documents=texts_eco,
    collection_name="economic_data",
    embedding=hg_embeddings,
    persist_directory=persist_directory
)

In [ ]:
question = "Microsoft(MSFT)"
docs_eco = economic_langchain_chroma.similarity_search(question,k=3)

In [ ]:
!pip install ipython

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_api_token"

llm = HuggingFaceHub(
    repo_id="tiiuae/falcon-7b-instruct",
    model_kwargs={"temperature": 0.1},
)

retriever_eco = economic_langchain_chroma.as_retriever(search_kwargs={"k":2})
qs="Microsoft(MSFT) Financial Report"
template = """You are a Financial Market Expert and Get the Market Economic Data and Market News about Company and Build the Financial Report for me.
              Understand this Market Information {context} and Answer the Query for this Company {question}. i just need the data into Tabular Form as well."""

PROMPT = PromptTemplate(input_variables=["context","question"], template=template)
qa_with_sources = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",chain_type_kwargs = {"prompt": PROMPT}, retriever=retriever_eco, return_source_documents=True)
llm_response = qa_with_sources({"query": qs})

In [ ]:
Markdown(llm_response['result'])

You are a Financial Market Expert and Get the Market Economic Data and Market News about Company and Build the Financial Report for me.
              Understand this Market Information : 0
symbol: MSFT
name: Microsoft Corporation

earningsAnnouncement: 2025-04-23 20:00:00+00:00 and Answer the Query for this Company Microsoft(MSFT) Financial Report. i just need the data into Tabular Form as well.
<p>The following is the financial report for Microsoft Corporation (MSFT) for the year 2025. The report includes the following sections:</p>

<ul>
<li>Income Statement</li>
<li>Balance Sheet</li>
<li>Cash Flow Statement</li>
<li>Income Statement</li>
<li>Balance Sheet</li>
<li>Cash Flow Statement</li>
</ul>

<p>The following is the income statement for the year 2025:</p>

<ul>
<li>Revenue: $128,000,000,000</li>
<li>Net Income: $21,000,000,000</li>
</ul>

<p>The following is the balance sheet for the year 2025:</p>

<ul>
<li>Assets: $128,000,000,000</li>
<li>Liabilities: $128,000,000,000</li>
<li>Equity: $128,000,000,000</li>
</ul>

<p>The following is the cash flow statement for the year 2025:</p>

<ul>
<li>Net Cash Flow: $21,000,000,000</li>
</ul>

<p>The following is the income statement for the year 2025:</p>

<ul>
<li>Revenue: $128,000,000,000</li>
<li>Net Income: $21,000,000,000</li>
</ul>

<p>The following is the balance sheet for the year 2025:</p>

<ul>
<li>Assets: $128,000,000,000</li>
<li>Liabilities: $128,000,000,000</li>
<li>Equity: $128,000,000,000</li>
</ul>

<p>The following is the cash flow statement for the year 2025:</p>

<ul>
<li>Net Cash Flow: $21,000,000,000</li>
</ul>

<p>The following is the income statement for the year 2025:</p>

<ul>
<li>Revenue: $128,000,000,000</li>
<li>Net Income: $21,000,000,000</li>
</ul>

In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.6 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
import pandas as pd

def convert_to_pdf(data, filename="Financial_Report.pdf"):
    pdf = SimpleDocTemplate(filename, pagesize=A4)
    elements = []

    # Title Style
    styles = getSampleStyleSheet()
    title = Paragraph("<b>Financial Report</b>", styles['Title'])
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Convert Data to Table Format
    if isinstance(data, pd.DataFrame):
        table_data = [data.columns.tolist()] + data.values.tolist()
    else:
        table_data = [list(data[0].keys())] + [list(row.values()) for row in data]

    table = Table(table_data)

    # Table Styling
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    elements.append(table)

    # Build PDF
    pdf.build(elements)
    print(f"PDF generated successfully: {filename}")

# Updated Sample Data (Formatted for clarity)
sample_data = pd.DataFrame({
    "Metric": ["Revenue", "Net Income", "EPS", "P/E Ratio", "Dividend Yield", "Market Cap",
               "Enterprise Value", "EV/EBITDA", "Debt/Equity", "Return on Assets", "Return on Equity",
               "Price", "52-Week Range", "Volume", "Average Volume", "Shares Outstanding",
               "Insider Ownership", "Institutional Ownership"],
    "Value": ["168.09B", "61.27B", "2.56", "30.93", "0.83%", "2.52T",
               "2.77T", "31.28", "0.27", "16.27%", "27.44%",
               "265.69", "209.19 - 349.66", "25.34M", "35.11M", "9.54B",
               "0.10%", "71.40%"],
    "Period": ["YTD", "YTD", "YTD", "YTD", "YTD", "04/01/2023",
                "04/01/2023", "YTD", "YTD", "YTD", "YTD",
                "04/01/2023", "", "04/01/2023", "3-Month", "04/01/2023",
                "04/01/2023", "04/01/2023"]
})

convert_to_pdf(sample_data)


PDF generated successfully: Financial_Report.pdf


In [ ]:
from google.colab import files
files.download("Financial_Report.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>